In [3]:
!pip install -q trafilatura pyyaml tqdm python-dotenv


In [4]:
import os,pathlib,textwrap,yaml
ROOT=pathlib.Path.cwd()
ART_HTML =ROOT /"artifacts"/"html"
ART_MD= ROOT/" artifacts"/"md"
REG_DIR =ROOT/"registry"
for p in [ART_HTML,ART_MD,REG_DIR]:
    p.mkdir(parents=True,exist_ok=True)
sources={
    "seeds":[
        "https://surfer.nmr.mgh.harvard.edu/fswiki/reckon-all",
        "https://fmriprep.org/en/stable/usage.html"
    ]
}
with open(REG_DIR /"sources.yaml","w",encoding="utf-8") as f:
    yaml.safe_dump(sources,f,sort_keys=False)
ART_HTML,ART_MD,REG_DIR
          

(WindowsPath('C:/Users/Patron/neuro-assitant/artifacts/html'),
 WindowsPath('C:/Users/Patron/neuro-assitant/ artifacts/md'),
 WindowsPath('C:/Users/Patron/neuro-assitant/registry'))

In [6]:
import hashlib,requests,trafilatura
from urllib.parse import urlparse

HEADERS={"USER-Agent":"neuro-assistant/0.1(local-notebook)"}
def sha(text:str)->str:
    return hashlib.sha256(text.encode("utf-8",errors="ignore")).hexdigest()[:12]

def fetch(url:str)-> str:
    r =requests.get(url,timeout=30,headers=HEADERS)
    r.raise_for_status()
    return r.text

def to_markdown(html:str,url:str)->str:
    txt =trafilatura.extract(html, url=url, include_links=True, include_tables=True,include_formatting=True)
    return txt or ""

def safe_filename(url:str)->str:
    u=url.replace("https://","").replace("http://","")
    return u.replace("/","_").replace("?","_").replace("#","_")

with open(REG_DIR /"sources.yaml","r",encoding="utf-8")as f:
    seed_url =yaml.safe_load(f)["seeds"][0]
    
html = fetch(seed_url)
md = to_markdown(html,seed_url)
tag= sha(seed_url)
(ART_HTML /f"{tag}.html").write_text(html,encoding=" utf-8")
(ART_MD /f"{safe_filename(seed_url)}.md").write_text(md, encoding=" utf-8")
print("Saved HTML ->",ART_HTML/f"{tag}.html")
print("Saved MD ->",ART_MD /f"{safe_filename(seed_url)}.md")
print("\n preview(first 500 chars):\n",md[:500])

HTTPError: 404 Client Error: NOTFOUND for url: https://surfer.nmr.mgh.harvard.edu/fswiki/reckon-all